In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from IPython.display import Image
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from shutil import copy2
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [2]:
pokemon_df = pd.read_csv('pokemon.csv')
pokemon_df

,Name,Type1,Type2
0,bulbasaur,Grass,Poison
1,ivysaur,Grass,Poison
2,venusaur,Grass,Poison
3,charmander,Fire,NaN
4,charmeleon,Fire,NaN
...,...,...,...
804,stakataka,Rock,Steel
805,blacephalon,Fire,Ghost
806,zeraora,Electric,NaN
807,meltan,Steel,NaN


In [3]:
pokemon_imgs = os.listdir('images')

sorted_imgs = pokemon_imgs.copy()
for pokemon in pokemon_imgs:
    pokemon_name = pokemon.split('.')[0]
    index = pokemon_df[pokemon_df.Name == pokemon_name].index.values[0]
    sorted_imgs[index] = pokemon

In [4]:
pokemon_df['Pokemon Image'] = sorted_imgs

In [5]:
if not os.path.exists('train/'):
    os.mkdir('train/')
if not os.path.exists('val/'):
    os.mkdir('val/')

In [6]:
for pk_type in pokemon_df.Type1.unique():
    if not os.path.exists(f'train/{pk_type}/'):
        os.mkdir('train/'+str(pk_type)+'/')
    if not os.path.exists(f'val/{pk_type}/'):
        os.mkdir('val/'+str(pk_type)+'/')

In [7]:
X_train, X_val, y_train, y_val = train_test_split(pokemon_df['Pokemon Image'], pokemon_df.Type1, 
                                                    test_size=0.3, stratify=pokemon_df.Type1, random_state=42)

In [8]:
for from_path, to_path in zip(X_train.values, y_train.values):
    if not os.path.exists( f'train/{to_path}'):
        copy2(f'images/{from_path}', f'train/{to_path}')
    

In [10]:
for from_path, to_path in zip(X_val.values, y_val.values):
    if not os.path.exists( f'val/{to_path}'):
        copy2(f'images/{from_path}', f'val/{to_path}')

In [11]:
train_datagen = ImageDataGenerator().flow_from_directory('train/')
val_datagen = ImageDataGenerator().flow_from_directory('val/')

Found 566 images belonging to 18 classes.
Found 243 images belonging to 18 classes.


In [12]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (256,256, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3, 3)))
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3, 3)))
classifier.add(Conv2D(128, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3, 3)))
classifier.add(Flatten())
classifier.add(Dense(units = 512, activation = 'relu'))
classifier.add(Dense(units = 18, activation = 'sigmoid'))

classifier.compile(loss='categorical_crossentropy', optimizer= "adam", metrics=['accuracy'])

In [ ]:
classifier.fit(train_datagen,
               epochs = 5,
               validation_data = val_datagen)

C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/5
18/18 [==============================] - 16s 894ms/step - loss: 60.7140 - accuracy: 0.0493 - val_loss: 2.8501 - val_accuracy: 0.1358
Epoch 2/5
18/18 [==============================] - 11s 620ms/step - loss: 2.6374 - accuracy: 0.1824 - val_loss: 2.7572 - val_accuracy: 0.1399
Epoch 3/5
18/18 [==============================] - 12s 651ms/step - loss: 2.3488 - accuracy: 0.2937 - val_loss: 2.8174 - val_accuracy: 0.1358
Epoch 4/5
 5/18 [=======>......................] - ETA: 6s - loss: 1.6321 - accuracy: 0.5894